<a href="https://colab.research.google.com/github/zebular13/Ambassador_Agreement/blob/master/Retrain_EfficientDet_Lite_detector_to_recognize_Chickens.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### *Copyright 2021 Google LLC*
*Licensed under the Apache License, Version 2.0 (the "License")*

In [ ]:
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Retrain EfficientDet for the Edge TPU with TensorFlow Lite Model Maker

In this tutorial, we'll retrain the EfficientDet-Lite object detection model (derived from [EfficientDet](https://ai.googleblog.com/2020/04/efficientdet-towards-scalable-and.html)) using the [TensorFlow Lite Model Maker library](https://www.tensorflow.org/lite/guide/model_maker).

We'll retrain the model using a publicly available dataset of chicken photos, teaching the model to recognize a chicken by name.




To start running all the code in this tutorial, select **Runtime > Run all** in the Colab toolbar.

## Import the required packages

Import [TensorFlow Lite Model Maker](https://www.tensorflow.org/lite/guide/model_maker). 
We'll also be using numpy and os packages. 
Finally, i mount google drive (this is where my images are stored). It will provide a link that you must follow to sign into google drive. Copy the code here and paste it back into the field that will show up in this notebook. 

In [ ]:
!pip install -q tflite-model-maker

In [ ]:
import numpy as np
import os

from tflite_model_maker.config import ExportFormat
from tflite_model_maker import model_spec
from tflite_model_maker import object_detector

import tensorflow as tf
assert tf.__version__.startswith('2')

tf.get_logger().setLevel('ERROR')
from absl import logging
logging.set_verbosity(logging.ERROR)
from google.colab import drive 
drive.mount('/content/gdrive')

## Load the training data


The chicken dataset includes labels to identify four individual chickens:
"Angel", "Broccoli", "Angel Baby" and "LadyFeathers MgGee".

Model Maker requires that we load our dataset using the [`DataLoader`](https://www.tensorflow.org/lite/api_docs/python/tflite_model_maker/object_detector/DataLoader) API. So in this case, we'll load it from a CSV file in google drive that defines 2384 images for training, 300 images for validation, and 300 images for testing. 


In [ ]:
train_data, validation_data, test_data = object_detector.DataLoader.from_csv('gdrive/MyDrive/chickens3/chickens-8.csv')

The CSV is in this format:

|| Asssigned Set | Image path | Label | xmin | ymin  | xmax | ymax|
|-|--------------|-----------|---------------|--------|-------|------|------|
||  TRAIN        | gdrive/MyDrive/chickens3/chickens2-mp4-2565.jpg       | Broccoli        | 487      | 3 | 636 | 350 |
||  TRAIN        | gdrive/MyDrive/chickens3/chickens2-mp4-2565.jpg       | Ladyfeathers MgGee   | 428    | 25 |  545 | 476 |
||  TRAIN   | gdrive/MyDrive/chickens3/chickens2-mp4-2565.jpg       | Angel          | 296               |15  | 636| 350|
||  VALIDATION         | gdrive/MyDrive/chickens3/chickens2-mp4-2567.jpg       | Broccoli           | 487               | 3 | 636 | 350|
||  TEST         | gdrive/MyDrive/chickens3/chickens2-mp4-2570.jpg       | Broccoli           | 486               | 1| 631 | 349 |
||  TEST         | gdrive/MyDrive/chickens3/chickens2-mp4-2570.jpg       | Angel Baby           | 189               | 347  | 543| 476 |
||

If you want to load your own dataset as a CSV file, you can learn more about the format in [Formatting a training data CSV](https://cloud.google.com/vision/automl/object-detection/docs/csv-format). You can load your CSV either from [Cloud Storage](https://cloud.google.com/storage) (as shown above) or from a local path.

[`DataLoader`](https://www.tensorflow.org/lite/api_docs/python/tflite_model_maker/object_detector/DataLoader) can also load your dataset in other formats, such as from a set of TFRecord files or from a local directory using the PASCAL VOC format. 

## Select the model spec

Model Maker supports the EfficientDet-Lite family of object detection models. (EfficientDet-Lite is derived from [EfficientDet](https://ai.googleblog.com/2020/04/efficientdet-towards-scalable-and.html), which offer state-of-the-art accuracy in a small model size). There are several model sizes you can choose from:

|| Model architecture | Size(MB)* | Latency(ms)** | Average Precision*** |
|-|--------------------|-----------|---------------|----------------------|
|| EfficientDet-Lite0 | 4.4       | 37            | 25.69%               |
|| EfficientDet-Lite1 | 5.8       | 49            | 30.55%               |
|| EfficientDet-Lite2 | 7.2       | 69            | 33.97%               |
|| EfficientDet-Lite3 | 11.4      | 116           | 37.70%               |
|| EfficientDet-Lite4 | 19.9      | 260           | 41.96%               |
| <td colspan=4><br><i>* File size of the integer quantized models. <br/>** Latency measured on Pixel 4 using 4 threads on CPU. <br/>*** Average Precision is the mAP (mean Average Precision) on the COCO 2017 validation dataset.</i></td> |

Beware that the bigger models (Lite3 and Lite4) do not fit onto the Edge TPU's onboard memory, so you'll see even greater latency when using those due to the cost of fetching data from the host system memory. Maybe this extra latency is okay for your application, but if it's not and you require the precision of the larger models, then you can [pipeline the model across multiple Edge TPUs](https://coral.ai/docs/edgetpu/pipeline/) (more about this when we compile the model below).

For this tutorial, we'll use Lite0:

In [ ]:
spec = object_detector.EfficientDetLite4Spec()

The [`EfficientDetLite0Spec`](https://www.tensorflow.org/lite/api_docs/python/tflite_model_maker/object_detector/EfficientDetLite0Spec) constructor also supports several arguments that specify training options, such as the max number of detections (default is 25 for the TF Lite model) and whether to use Cloud TPUs for training. You can also use the constructor to specify the number of training epochs and the batch size, but you can also specify those in the next step.

## Create and train the model

Now we need to create our model according to the model spec, load our dataset into the model, specify training parameters, and begin training. 

Using Model Maker, we accomplished all of that with [`create()`](https://www.tensorflow.org/lite/api_docs/python/tflite_model_maker/object_detector/create):

In [ ]:
model = object_detector.create(train_data=train_data, 
                               model_spec=spec, 
                               validation_data=validation_data, 
                               epochs=50, 
                               batch_size=32, 
                               train_whole_model=True)

## Evaluate the model

Now we'll use the remaining 300 images in our test dataset to evaluate how well the model performs with data it has never seen before.

The default batch size for [EfficientDetLite models](https://www.tensorflow.org/lite/api_docs/python/tflite_model_maker/object_detector/EfficientDetSpec) is 64. To make sure it can go through all images in one step, you can specify the `batch_size` argument to 300 when you call [`evaluate()`](https://www.tensorflow.org/lite/api_docs/python/tflite_model_maker/object_detector/ObjectDetector#evaluate).

In [ ]:
model.evaluate(test_data,
               batch_size=300)

## What do these metrics mean?

The [`evaluate()`](https://www.tensorflow.org/lite/api_docs/python/tflite_model_maker/object_detector/ObjectDetector#evaluate) method provides output in the style of [COCO evaluation metrics](https://cocodataset.org/#detection-eval):


***Average Precision:***
* 'AP':  Average Precision 
* 'AP50': AP at IoU=.50 (PASCAL VOC metric)
* 'AP75': AP at IoU=.75 (strict metric)

***Average Precision for each Label:***
* 'AP_/Label_x': 

***Average Precision Across Scales:***
* 'APl': AP for large objects: area > 96 ** 2
* 'APm': AP for medium objects: 322 < area < 96 ** 2
* 'APs':  AP for small objects: area < 32 ** 2

***Average Recall***
* 'ARl': 
* 'ARm': 
* 'ARmax1': 
* 'ARmax10':
* 'ARmax100': 
* 'ARs': 




## Export to TensorFlow Lite

Next, we'll export the model to the TensorFlow Lite format. By default, the [`export()`](https://www.tensorflow.org/lite/api_docs/python/tflite_model_maker/object_detector/ObjectDetector#export) method performs [full integer post-training quantization](https://www.tensorflow.org/lite/performance/post_training_quantization#full_integer_quantization), which is exactly what we need for compatibility with the Edge TPU. (Model Maker uses the same dataset we gave to our model spec as a representative dataset, which is required for full-int quantization.)

We just need to specify the export directory and format. By default, it exports to TF Lite, but we also want a labels file, so we declare both:

In [ ]:
model.export(export_dir='.',
             tflite_filename='efficientdet-lite-chickens.tflite',
             label_filename='chicken-labels.txt',
             export_format=[ExportFormat.TFLITE, ExportFormat.LABEL])

### Evaluate the TF Lite model

Exporting the model to TensorFlow Lite can affect the model accuracy, due to the reduced numerical precision from quantization and because the original TensorFlow model uses per-class [non-max supression (NMS)](https://www.coursera.org/lecture/convolutional-neural-networks/non-max-suppression-dvrjH) for post-processing, while the TF Lite model uses global NMS, which is faster but less accurate.

Therefore you should always evaluate the exported TF Lite model and be sure it still meets your requirements:

In [ ]:
model.evaluate_tflite('efficientdet-lite-chickens.tflite', test_data)

### Test it on a new image

Just to be sure of things, let's run an inference with the TF Lite model ourselves. 

To simplify our code, we'll use the [PyCoral API](https://coral.ai/docs/reference/py/):

In [ ]:
! python3 -m pip install --extra-index-url https://google-coral.github.io/py-repo/ pycoral

In [ ]:
INPUT_IMAGE = 'gdrive/MyDrive/chickens3/chickens2-mp4-47367.jpg'
#INPUT_IMAGE = 'gdrive/MyDrive/chickenspics/chickens-5549.jpeg'

# Set the model files
MODEL_FILE = 'efficientdet-lite-chickens.tflite'
LABELS_FILE = 'chicken-labels.txt'
DETECTION_THRESHOLD = 0.1

from PIL import Image
from PIL import ImageDraw
from PIL import ImageFont

import tflite_runtime.interpreter as tflite 
from pycoral.adapters import common
from pycoral.adapters import detect
from pycoral.utils.dataset import read_label_file

def draw_objects(draw, objs, labels):
  """Draws the bounding box and label for each object."""
  COLORS = np.random.randint(0, 255, size=(len(labels), 3), dtype=np.uint8)
  for obj in objs:
    bbox = obj.bbox
    color = tuple(int(c) for c in COLORS[obj.id])
    draw.rectangle([(bbox.xmin, bbox.ymin), (bbox.xmax, bbox.ymax)],
                   outline=color, width=15)
    font = ImageFont.truetype("LiberationSans-Regular.ttf", size=90)
    draw.text((bbox.xmin + 20, bbox.ymin + 20),
              '%s\n%.2f' % (labels.get(obj.id, obj.id), obj.score),
              fill=color, font=font)

# Load the TF Lite model
labels = read_label_file(LABELS_FILE)
interpreter = tflite.Interpreter(MODEL_FILE)
interpreter.allocate_tensors()

# Resize the image
image = Image.open(INPUT_IMAGE)
_, scale = common.set_resized_input(
    interpreter, image.size, lambda size: image.resize(size, Image.ANTIALIAS))

# Run inference and draw boxes
interpreter.invoke()
objs = detect.get_objects(interpreter, DETECTION_THRESHOLD, scale)
draw_objects(ImageDraw.Draw(image), objs, labels)

# Show the results
width = 400
height_ratio = image.height / image.width
image.resize((width, int(width * height_ratio)))

## Download the files

In [ ]:
from google.colab import files

files.download('efficientdet-lite-chickens.tflite')
files.download('efficientdet-lite-chickens_edgetpu.tflite')
files.download('chicken-labels.txt')

## More resources

* For more information about the Model Maker library used in this tutorial, see the [TensorFlow Lite Model Maker guide](https://www.tensorflow.org/lite/guide/model_maker) and [API reference](https://www.tensorflow.org/lite/api_docs/python/tflite_model_maker).

* For other transfer learning tutorials that are compatible with the Edge TPU, see the [Colab tutorials for Coral](https://github.com/google-coral/tutorials#colab-tutorials-for-coral).

* You can also find more examples that show how to run inference on the Edge TPU at [coral.ai/examples](https://coral.ai/examples/#code-examples/).